# Prediksi Klaim Asuransi dengan Chronos-2
Forecasting frekuensi klaim, total klaim, dan severitas untuk 5 bulan ke depan

## Tahap 1: Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
from darts import TimeSeries, concatenate
from darts.models import Chronos2Model
from darts.dataprocessing.transformers import Scaler
import warnings
import os
import torch
from mlforecast import MLForecast
# from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

warnings.filterwarnings("ignore")

/mnt/data/lomba/mcf/pengerjaan/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The StatsForecast module could not be imported. To enable support for the AutoARIMA, AutoETS and Croston models, please consider installing it.
The `XGBoost` module could not be imported. To enable XGBoost support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md
The `XGBoost` module could not be imported. To enable XGBoost support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md


In [2]:
# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("GPU not available. Training will use CPU.")

Using device: cuda
GPU Name: NVIDIA GeForce RTX 3050 Ti Laptop GPU
GPU Memory: 3.95 GB
CUDA Version: 13.1


## Tahap 2: Load and Merge Datasets

In [3]:
# Load Datasets
print("Loading datasets...")
df_polis = pd.read_csv("../dataset/Data_Polis.csv")
df_klaim = pd.read_csv("../dataset/Data_Klaim.csv")

print(f"Data Polis shape: {df_polis.shape}")
print(f"Data Klaim shape: {df_klaim.shape}")

# Identify Keys & Merge
print("\nMerging data...")
df = pd.merge(df_klaim, df_polis, on="Nomor Polis", how="left")
print(f"Merged data shape: {df.shape}")

Loading datasets...
Data Polis shape: (4096, 6)
Data Klaim shape: (4627, 13)

Merging data...
Merged data shape: (4627, 18)


## Tahap 3: Clean Date Columns

In [4]:
# Data Cleaning (Dates)
print("Cleaning dates...")
date_columns = ["Tanggal Pembayaran Klaim", "Tanggal Pasien Masuk RS", "Tanggal Pasien Keluar RS", "Tanggal Efektif Polis"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

df["Tanggal Lahir"] = pd.to_datetime(df["Tanggal Lahir"], format="%Y%m%d", errors='coerce')

print("Date conversion completed")

Cleaning dates...
Date conversion completed


## Tahap 4: Filter Data by Date Range

In [5]:
# Analysis Population (Filter Date Range)
start_date = "2024-01-01"
end_date = "2025-07-31"
df_filtered = df[(df["Tanggal Pasien Masuk RS"] >= start_date) & 
                 (df["Tanggal Pasien Masuk RS"] <= end_date)].copy()

print(f"Total Claims after filtering: {len(df_filtered)}")
print(f"Date range: {start_date} to {end_date}")

Total Claims after filtering: 4627
Date range: 2024-01-01 to 2025-07-31


## Tahap 5: Feature Engineering - Age and Region

In [6]:
# Profil Risiko: Usia & Wilayah
reference_date = pd.to_datetime("2025-08-01")
df_filtered["Usia"] = (reference_date - df_filtered["Tanggal Lahir"]).dt.days // 365
df_filtered["Wilayah"] = df_filtered["Plan Code"].map({"M-001": "Wilayah A", "M-002": "Wilayah B", "M-003": "Wilayah C"})

print("Age and Region features created")
print(f"\nAge statistics:\n{df_filtered['Usia'].describe()}")

Age and Region features created

Age statistics:
count    4627.000000
mean       59.719257
std        12.569963
min         8.000000
25%        52.000000
50%        60.000000
75%        69.000000
max        91.000000
Name: Usia, dtype: float64


## Tahap 6: Feature Engineering - ICD Grouping and Duration

In [7]:
# Kategorisasi Medis: ICD Grouping
df_filtered["ICD_Group"] = df_filtered["ICD Diagnosis"].astype(str).str.split('.').str[0]

# Durasi dan Metode
df_filtered["Durasi_Rawat"] = (df_filtered["Tanggal Pasien Keluar RS"] - df_filtered["Tanggal Pasien Masuk RS"]).dt.days
df_filtered["Durasi_Rawat"] = df_filtered["Durasi_Rawat"].clip(lower=0)

print("ICD Grouping and Duration features created")
print(f"\nDuration of stay statistics:\n{df_filtered['Durasi_Rawat'].describe()}")

ICD Grouping and Duration features created

Duration of stay statistics:
count    4627.000000
mean        1.264102
std         2.930572
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max        54.000000
Name: Durasi_Rawat, dtype: float64


## Tahap 7: Aggregate Historical Metrics by Month

In [8]:
df_filtered["Bulan"] = df_filtered["Tanggal Pasien Masuk RS"].dt.to_period("M")

monthly_agg = df_filtered.groupby("Bulan").agg(
    Frequency=("Claim ID", "nunique"),
    Total_Claim=("Nominal Klaim Yang Disetujui", "sum")
).reset_index()

monthly_agg["Severity"] = monthly_agg["Total_Claim"] / monthly_agg["Frequency"]
monthly_agg["Bulan_Ts"] = monthly_agg["Bulan"].dt.to_timestamp()

print("Historical Metrics:")
print(monthly_agg)

Historical Metrics:
      Bulan  Frequency   Total_Claim      Severity   Bulan_Ts
0   2024-01        302  2.026098e+10  6.708934e+07 2024-01-01
1   2024-02        208  1.385965e+10  6.663291e+07 2024-02-01
2   2024-03        278  1.431126e+10  5.147935e+07 2024-03-01
3   2024-04        239  1.144106e+10  4.787056e+07 2024-04-01
4   2024-05        263  1.221146e+10  4.643141e+07 2024-05-01
5   2024-06        225  1.212517e+10  5.388963e+07 2024-06-01
6   2024-07        257  1.497052e+10  5.825104e+07 2024-07-01
7   2024-08        228  1.351294e+10  5.926726e+07 2024-08-01
8   2024-09        208  1.226412e+10  5.896211e+07 2024-09-01
9   2024-10        274  1.268117e+10  4.628163e+07 2024-10-01
10  2024-11        270  1.373306e+10  5.086318e+07 2024-11-01
11  2024-12        238  1.201391e+10  5.047861e+07 2024-12-01
12  2025-01        216  9.610380e+09  4.449250e+07 2025-01-01
13  2025-02        246  1.748054e+10  7.105911e+07 2025-02-01
14  2025-03        230  1.367924e+10  5.947496e+07

## Tahap 8: Create Time Series Objects

In [9]:
ts_freq = TimeSeries.from_dataframe(monthly_agg, "Bulan_Ts", "Frequency")
ts_total = TimeSeries.from_dataframe(monthly_agg, "Bulan_Ts", "Total_Claim")
ts_sev = TimeSeries.from_dataframe(monthly_agg, "Bulan_Ts", "Severity")

print("Time Series objects created:")
print(f"Frequency series length: {len(ts_freq)}")
print(f"Total Claim series length: {len(ts_total)}")
print(f"Severity series length: {len(ts_sev)}")

Time Series objects created:
Frequency series length: 19
Total Claim series length: 19
Severity series length: 19


## Tahap 9: Define Forecast Function

In [19]:
# Chronos Model Setup dengan GPU Support
model_name = "amazon/chronos-2"  
input_chunk_length = 8  # Reduced from 12 for backtesting compatibility
output_chunk_length = 5 # Forecast 5 months

# GPU Configuration
pl_trainer_kwargs = {}
if torch.cuda.is_available():
    pl_trainer_kwargs = {
        "accelerator": "gpu",
        "devices": [0],  # Use first GPU
        "precision": 32   # Use float32 for stability
    }
    print(f"✓ GPU enabled: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available, using CPU")

print(f"Model: {model_name}")
print(f"Input chunk length: {input_chunk_length} months")
print(f"Output chunk length: {output_chunk_length} months")
print(f"Min series length required: {input_chunk_length + output_chunk_length} months")

def forecast_series(series, model_name, input_len, output_len, pl_trainer_kwargs):
    model = Chronos2Model(
        input_chunk_length=input_len,
        output_chunk_length=output_len,
        hub_model_name=model_name,
        random_state=42,
        pl_trainer_kwargs=pl_trainer_kwargs  # Pass GPU config here
    )
    
    print(f"Training model...")
    model.fit(series, verbose=True)
    prediction = model.predict(output_len)
    return prediction

✓ GPU enabled: NVIDIA GeForce RTX 3050 Ti Laptop GPU
Model: amazon/chronos-2
Input chunk length: 8 months
Output chunk length: 5 months
Min series length required: 13 months


## Tahap 10: Forecast Claim Frequency

In [20]:
print("Forecasting Frequency...")
pred_freq = forecast_series(ts_freq, model_name, input_chunk_length, output_chunk_length, pl_trainer_kwargs)
print("✓ Frequency forecast completed")

# Clear GPU cache if using CUDA
if torch.cuda.is_available():
    torch.cuda.empty_cache()

Forecasting Frequency...
Training model...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LO

✓ Frequency forecast completed


## Tahap 11: Forecast Total Claim Amount

In [21]:
print("Forecasting Total Claim...")
pred_total = forecast_series(ts_total, model_name, input_chunk_length, output_chunk_length, pl_trainer_kwargs)
print("✓ Total Claim forecast completed")

# Clear GPU cache if using CUDA
if torch.cuda.is_available():
    torch.cuda.empty_cache()

Forecasting Total Claim...
Training model...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


✓ Total Claim forecast completed


## Tahap 12: Forecast Claim Severity

In [22]:
print("Forecasting Severity...")
pred_sev = forecast_series(ts_sev, model_name, input_chunk_length, output_chunk_length, pl_trainer_kwargs)
print("✓ Severity forecast completed")

# Clear GPU cache if using CUDA
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✓ GPU memory cleared")

Forecasting Severity...
Training model...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


✓ Severity forecast completed
✓ GPU memory cleared


## Tahap 13: Model Evaluation (Backtesting)

In [23]:
# Define MAPE function
def calculate_mape(actual, predicted):
    """
    Calculate Mean Absolute Percentage Error (MAPE)
    """
    actual = np.array(actual).flatten()
    predicted = np.array(predicted).flatten()
    mask = actual != 0
    mape = np.mean(np.abs((actual[mask] - predicted[mask]) / actual[mask])) * 100
    return mape

print("✓ MAPE function defined")

✓ MAPE function defined


In [24]:
# Split data untuk backtesting (5 bulan terakhir = test set)
train_size = len(monthly_agg) - 5
test_size = 5

monthly_train = monthly_agg.iloc[:train_size].copy()
monthly_test = monthly_agg.iloc[train_size:].copy()

print(f"Train: {monthly_train['Bulan'].iloc[0]} to {monthly_train['Bulan'].iloc[-1]}")
print(f"Test:  {monthly_test['Bulan'].iloc[0]} to {monthly_test['Bulan'].iloc[-1]}")

Train: 2024-01 to 2025-02
Test:  2025-03 to 2025-07


In [25]:
# Create time series dari train data
monthly_train['Bulan_Ts'] = monthly_train['Bulan'].dt.to_timestamp()

ts_freq_train = TimeSeries.from_dataframe(monthly_train, "Bulan_Ts", "Frequency")
ts_total_train = TimeSeries.from_dataframe(monthly_train, "Bulan_Ts", "Total_Claim")
ts_sev_train = TimeSeries.from_dataframe(monthly_train, "Bulan_Ts", "Severity")

print("✓ Train time series created")

✓ Train time series created


In [26]:
# Forecast test period
pred_freq_test = forecast_series(ts_freq_train, model_name, input_chunk_length, test_size, pl_trainer_kwargs)
pred_total_test = forecast_series(ts_total_train, model_name, input_chunk_length, test_size, pl_trainer_kwargs)
pred_sev_test = forecast_series(ts_sev_train, model_name, input_chunk_length, test_size, pl_trainer_kwargs)

if torch.cuda.is_available():
    torch.cuda.empty_cache()

Training model...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training model...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training model...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [27]:
# Calculate MAPE
actual_freq = monthly_test['Frequency'].values
predicted_freq = pred_freq_test.values().flatten()
mape_frequency = calculate_mape(actual_freq, predicted_freq)

actual_total = monthly_test['Total_Claim'].values
predicted_total = pred_total_test.values().flatten()
mape_total = calculate_mape(actual_total, predicted_total)

actual_severity = monthly_test['Severity'].values
predicted_severity = pred_sev_test.values().flatten()
mape_severity = calculate_mape(actual_severity, predicted_severity)

avg_mape = np.mean([mape_frequency, mape_severity, mape_total])

print("\n" + "="*60)
print("MODEL EVALUATION (Backtesting)")
print("="*60)
print(f"✓ MAPE Frekuensi:   {mape_frequency:.2f}%")
print(f"✓ MAPE Severitas:   {mape_severity:.2f}%")
print(f"✓ MAPE Total:       {mape_total:.2f}%")
print("-"*60)
print(f"✓ Rata-rata MAPE:   {avg_mape:.2f}%")
print("="*60)


MODEL EVALUATION (Backtesting)
✓ MAPE Frekuensi:   6.48%
✓ MAPE Severitas:   6.03%
✓ MAPE Total:       9.75%
------------------------------------------------------------
✓ Rata-rata MAPE:   7.42%


In [28]:
submission_list = []
forecast_dates = pd.date_range(start="2025-08-01", periods=5, freq="MS")

# Gunakan pred_freq, pred_total, pred_sev dari Tahap 10-12
freq_values = pred_freq.values()
total_values = pred_total.values()
sev_values = pred_sev.values()

for i, date in enumerate(forecast_dates):
    date_str = date.strftime("%Y_%m")
    
    submission_list.append({
        "id": f"{date_str}_Claim_Frequency",
        "value": freq_values[i][0]
    })
    
    submission_list.append({
        "id": f"{date_str}_Claim_Severity",
        "value": sev_values[i][0]
    })
    
    submission_list.append({
        "id": f"{date_str}_Total_Claim",
        "value": total_values[i][0]
    })

submission_df = pd.DataFrame(submission_list)

print(f"✓ Submission DataFrame shape: {submission_df.shape}")
print(submission_df)

✓ Submission DataFrame shape: (15, 2)
                         id         value
0   2025_08_Claim_Frequency  2.472533e+02
1    2025_08_Claim_Severity  5.244938e+07
2       2025_08_Total_Claim  1.328346e+10
3   2025_09_Claim_Frequency  2.449465e+02
4    2025_09_Claim_Severity  5.282429e+07
5       2025_09_Total_Claim  1.322625e+10
6   2025_10_Claim_Frequency  2.438276e+02
7    2025_10_Claim_Severity  5.289457e+07
8       2025_10_Total_Claim  1.321587e+10
9   2025_11_Claim_Frequency  2.429576e+02
10   2025_11_Claim_Severity  5.283902e+07
11      2025_11_Total_Claim  1.312850e+10
12  2025_12_Claim_Frequency  2.422160e+02
13   2025_12_Claim_Severity  5.269962e+07
14      2025_12_Total_Claim  1.305097e+10


In [ ]:
# Verify rows
if len(submission_df) != 15:
    print(f"Warning: Expected 15 rows, got {len(submission_df)}")
else:
    print("✓ Submission has exactly 15 rows")

# Output
output_path = "../submission/submission_chronos2.csv"
submission_df.to_csv(output_path, index=False)
print(f"\n✓ Submission saved to {output_path}")

## Tahap 16: MLForecast Model Comparison

In [ ]:
# Prepare data for MLForecast (format: unique_id, ds, y)
print("\nPreparing data for MLForecast...")

df_mlforecast_freq = pd.DataFrame({
    'unique_id': 'Frequency',
    'ds': monthly_agg['Bulan_Ts'],
    'y': monthly_agg['Frequency'].values
})

df_mlforecast_total = pd.DataFrame({
    'unique_id': 'Total_Claim',
    'ds': monthly_agg['Bulan_Ts'],
    'y': monthly_agg['Total_Claim'].values
})

df_mlforecast_sev = pd.DataFrame({
    'unique_id': 'Severity',
    'ds': monthly_agg['Bulan_Ts'],
    'y': monthly_agg['Severity'].values
})

# Combine all series
df_mlforecast = pd.concat([df_mlforecast_freq, df_mlforecast_total, df_mlforecast_sev], ignore_index=True)

print("Data shape for MLForecast:", df_mlforecast.shape)
print(df_mlforecast.head(10))

In [ ]:
# Train MLForecast with multiple models
print("\n" + "="*60)
print("TRAINING MLFORECAST MODELS")
print("="*60)

# Initialize MLForecast dengan XGBoost
fcst_xgb = MLForecast(
    models=[
        XGBRegressor(n_estimators=50, max_depth=5, learning_rate=0.1, random_state=42)
    ],
    freq='MS',  # Monthly start
    lags=[1, 3, 6],  # Use previous 1, 3, 6 months
    lag_transforms={
        1: [('mean', 3), ('std', 3)]  # 3-month rolling mean & std
    },
    num_threads=1
)

print("Training XGBRegressor...")
fcst_xgb.fit(df_mlforecast)
pred_mlforecast_xgb = fcst_xgb.predict(h=5, level=None)

print("✓ XGBoost training completed")
print("\nPredictions shape:", pred_mlforecast_xgb.shape)
print(pred_mlforecast_xgb)

In [ ]:
# Extract predictions per metric
pred_freq_mlf = pred_mlforecast_xgb[pred_mlforecast_xgb['unique_id'] == 'Frequency']['XGBRegressor'].values
pred_total_mlf = pred_mlforecast_xgb[pred_mlforecast_xgb['unique_id'] == 'Total_Claim']['XGBRegressor'].values
pred_sev_mlf = pred_mlforecast_xgb[pred_mlforecast_xgb['unique_id'] == 'Severity']['XGBRegressor'].values

# Calculate MAPE for MLForecast
mape_frequency_mlf = calculate_mape(actual_freq, pred_freq_mlf)
mape_severity_mlf = calculate_mape(actual_severity, pred_sev_mlf)
mape_total_mlf = calculate_mape(actual_total, pred_total_mlf)
avg_mape_mlf = np.mean([mape_frequency_mlf, mape_severity_mlf, mape_total_mlf])

print("\n" + "="*60)
print("MLFORECAST EVALUATION")
print("="*60)
print(f"✓ MAPE Frekuensi:   {mape_frequency_mlf:.2f}%")
print(f"✓ MAPE Severitas:   {mape_severity_mlf:.2f}%")
print(f"✓ MAPE Total:       {mape_total_mlf:.2f}%")
print("-"*60)
print(f"✓ Rata-rata MAPE:   {avg_mape_mlf:.2f}%")
print("="*60)

In [ ]:
# Compare Chronos-2 vs MLForecast
print("\n" + "="*80)
print("MODEL COMPARISON: CHRONOS-2 vs MLFORECAST")
print("="*80)

comparison_df = pd.DataFrame({
    'Metric': ['Frequency', 'Severity', 'Total_Claim', 'Average'],
    'Chronos-2 MAPE (%)': [
        mape_frequency,
        mape_severity,
        mape_total,
        avg_mape
    ],
    'MLForecast MAPE (%)': [
        mape_frequency_mlf,
        mape_severity_mlf,
        mape_total_mlf,
        avg_mape_mlf
    ]
})

comparison_df['Difference (%)'] = comparison_df['Chronos-2 MAPE (%)'] - comparison_df['MLForecast MAPE (%)']
comparison_df['Winner'] = comparison_df.apply(
    lambda row: 'MLForecast' if row['Difference (%)'] > 0 else 'Chronos-2', 
    axis=1
)

print(comparison_df.to_string(index=False))

# Best model
best_avg_mape = min(avg_mape, avg_mape_mlf)
best_model = 'Chronos-2' if avg_mape < avg_mape_mlf else 'MLForecast'
print("\n" + "-"*80)
print(f"✓ BEST MODEL: {best_model} (MAPE: {best_avg_mape:.2f}%)")
print("="*80)

In [ ]:
# Visualisasi comparison
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. MAPE Comparison - Frequency
ax = axes[0, 0]
models = ['Chronos-2', 'MLForecast']
freq_mapes = [mape_frequency, mape_frequency_mlf]
colors = ['#1f77b4' if mape_frequency < mape_frequency_mlf else '#ff7f0e',
          '#ff7f0e' if mape_frequency < mape_frequency_mlf else '#1f77b4']
ax.bar(models, freq_mapes, color=colors)
ax.set_ylabel('MAPE (%)', fontweight='bold')
ax.set_title('Frequency MAPE Comparison', fontweight='bold')
ax.grid(axis='y', alpha=0.3)
for i, v in enumerate(freq_mapes):
    ax.text(i, v + 0.5, f'{v:.2f}%', ha='center', fontweight='bold')

# 2. MAPE Comparison - Severity
ax = axes[0, 1]
sev_mapes = [mape_severity, mape_severity_mlf]
colors = ['#1f77b4' if mape_severity < mape_severity_mlf else '#ff7f0e',
          '#ff7f0e' if mape_severity < mape_severity_mlf else '#1f77b4']
ax.bar(models, sev_mapes, color=colors)
ax.set_ylabel('MAPE (%)', fontweight='bold')
ax.set_title('Severity MAPE Comparison', fontweight='bold')
ax.grid(axis='y', alpha=0.3)
for i, v in enumerate(sev_mapes):
    ax.text(i, v + 0.5, f'{v:.2f}%', ha='center', fontweight='bold')

# 3. MAPE Comparison - Total Claim
ax = axes[1, 0]
total_mapes = [mape_total, mape_total_mlf]
colors = ['#1f77b4' if mape_total < mape_total_mlf else '#ff7f0e',
          '#ff7f0e' if mape_total < mape_total_mlf else '#1f77b4']
ax.bar(models, total_mapes, color=colors)
ax.set_ylabel('MAPE (%)', fontweight='bold')
ax.set_title('Total Claim MAPE Comparison', fontweight='bold')
ax.grid(axis='y', alpha=0.3)
for i, v in enumerate(total_mapes):
    ax.text(i, v + 0.5, f'{v:.2f}%', ha='center', fontweight='bold')

# 4. Average MAPE Comparison
ax = axes[1, 1]
avg_mapes = [avg_mape, avg_mape_mlf]
colors = ['#2ca02c' if avg_mape < avg_mape_mlf else '#d62728',
          '#d62728' if avg_mape < avg_mape_mlf else '#2ca02c']
ax.bar(models, avg_mapes, color=colors, width=0.5)
ax.set_ylabel('Average MAPE (%)', fontweight='bold')
ax.set_title('Average MAPE Comparison (WINNER)', fontweight='bold', fontsize=12)
ax.set_ylim(0, max(avg_mapes) * 1.2)
ax.grid(axis='y', alpha=0.3)
for i, v in enumerate(avg_mapes):
    ax.text(i, v + 0.5, f'{v:.2f}%', ha='center', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.show()

print("✓ Comparison visualization completed")